In [1]:
import pandas as pd
import numpy as np

In [358]:
df = pd.read_csv("snli_1.0/snli_1.0_train.txt", sep="\t")

In [359]:
df.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( The church ) ( ( has ( cracks ( in ( the c...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ ha...,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,2677109430.jpg#1,2677109430.jpg#1r1n,neutral,contradiction,contradiction,neutral,neutral
1,entailment,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( The church ) ( ( is ( filled ( with song )...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ is...,This church choir sings to the masses as they ...,The church is filled with song.,2677109430.jpg#1,2677109430.jpg#1r1e,entailment,entailment,entailment,neutral,entailment
2,contradiction,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( ( A choir ) ( singing ( at ( a ( baseball ...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (NP (NP (DT A) (NN choir)) (VP (VBG sing...,This church choir sings to the masses as they ...,A choir singing at a baseball game.,2677109430.jpg#1,2677109430.jpg#1r1c,contradiction,contradiction,contradiction,contradiction,contradiction
3,neutral,( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,( ( The woman ) ( ( is young ) . ) ),(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...,"A woman with a green headscarf, blue shirt and...",The woman is young.,6160193920.jpg#4,6160193920.jpg#4r1n,neutral,neutral,neutral,neutral,neutral
4,entailment,( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,( ( The woman ) ( ( is ( very happy ) ) . ) ),(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,6160193920.jpg#4,6160193920.jpg#4r1e,entailment,entailment,contradiction,entailment,neutral


In [360]:
df = df[['sentence1', 'sentence2', 'label1']]
df.head()

,sentence1,sentence2,label1
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,neutral
1,This church choir sings to the masses as they ...,The church is filled with song.,entailment
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,contradiction
3,"A woman with a green headscarf, blue shirt and...",The woman is young.,neutral
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment


In [361]:
print(df.shape)
df = df.dropna()
df.shape

(10000, 3)


(10000, 3)

In [362]:
df = df.rename(columns={"sentence1": "premise", "sentence2": "hypothesis", "label1": "label"})
df.head()

,premise,hypothesis,label
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,neutral
1,This church choir sings to the masses as they ...,The church is filled with song.,entailment
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,contradiction
3,"A woman with a green headscarf, blue shirt and...",The woman is young.,neutral
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment


In [363]:
df["label"].value_counts()

neutral          3334
entailment       3333
contradiction    3333
Name: label, dtype: int64

In [364]:
# new_samples = df.shape[0] / (len(df["label"].unique()) + 1)
new_samples = df.shape[0] / 2
new_samples

5000.0

In [365]:
new_samples_per_label = int(new_samples / len(df["label"].unique()))
new_samples_per_label

1666

In [366]:
neutrals, contras, entails = df[df["label"] == "neutral"].sample(n=new_samples_per_label), df[df["label"] == "contradiction"].sample(n=new_samples_per_label), df[df["label"] == "entailment"].sample(n=new_samples_per_label)
print(neutrals.shape, contras.shape, entails.shape, df.shape)

(1666, 3) (1666, 3) (1666, 3) (10000, 3)


In [367]:
size = int(neutrals.shape[0] / 2)
size

833

In [368]:
neutrals_1, neutrals_2 = neutrals.iloc[0:size], neutrals.iloc[size:size * 2]
contras_1, contras_2 = contras.iloc[0:size], contras.iloc[size:size * 2]
entails_1, entails_2 = entails.iloc[0:size], entails.iloc[size:size * 2]

In [369]:
print(neutrals_1.shape, neutrals_2.shape)

(833, 3) (833, 3)


In [370]:
df_new = pd.DataFrame()
df_new = df_new.append(pd.DataFrame({"premise": list(neutrals_1["premise"]), "hypothesis": list(contras_2["hypothesis"])}))
df_new = df_new.append(pd.DataFrame({"premise": list(neutrals_2["premise"]), "hypothesis": list(entails_1["hypothesis"])}))
df_new = df_new.append(pd.DataFrame({"premise": list(contras_1["premise"]), "hypothesis": list(entails_2["hypothesis"])}))
df_new = df_new.append(pd.DataFrame({"premise": list(contras_2["premise"]), "hypothesis": list(neutrals_1["hypothesis"])}))
df_new = df_new.append(pd.DataFrame({"premise": list(entails_1["premise"]), "hypothesis": list(neutrals_2["hypothesis"])}))
df_new = df_new.append(pd.DataFrame({"premise": list(entails_2["premise"]), "hypothesis": list(contras_1["hypothesis"])}))
df_new["label"] = len(df["label"].unique())
df_new.shape

(4998, 3)

In [371]:
df_new.head()

,premise,hypothesis,label
0,several brides wearing white dresses are holdi...,Two married people avoid looking at one another.,3
1,A skateboard is grinding on a curb with his sk...,A man with short hair dancing outside while pl...,3
2,A girl in a red and white uniform is swinging ...,A man chops veggies.,3
3,people standing at a beach with Cameras.,3 people are seated with their backs against e...,3
4,Two friends having a laugh.,The woman is sleeping on a pillow.,3


In [372]:
df = df.drop(neutrals.index)
df.shape

(8334, 3)

In [373]:
df = df.drop(contras.index)
df.shape

(6668, 3)

In [374]:
df = df.drop(entails.index)
df.shape

(5002, 3)

In [375]:
df = df.append(df_new)
df.shape

(10000, 3)

In [376]:
df.loc[df["label"] == "entailment", "label"] = 0
df.loc[df["label"] == "neutral", "label"] = 1
df.loc[df["label"] == "contradiction", "label"] = 2

In [377]:
df["label"].value_counts()

3    4998
1    1668
0    1667
2    1667
Name: label, dtype: int64

In [378]:
df.to_csv("new_train_50.txt", sep="\t", index=False)